# Building an E-Model for a single-compartment cell

This notebook provides a simple example on how to run the pipeline to create an e-model of a single-compartment cell with two free parameters. In this instance, we will use rheobase independent optimisation. 

When the optimisation is rheobase dependent, it means that we need to calculate the rheobase value (the minimum current required to trigger a neuron) to normalise the trace values. In this case, the normalisation is done by expressing the trace values as a percentage of the rheobase. This approach ensures that the traces are scaled relative to the neuron’s threshold current, providing a consistent basis for comparison. 

However, here we will present the rheobase independent approach which means that we use specific amplitude values like `IDRest_1.0` (indicating a fixed current of 1.0 nA) instead of `IDRest_100` (which represents 100% of the rheobase).

For rheobase dependent optimisation or additional details, please refer to the [L5PC](./../L5PC/) example.

Please note that in this example we will use only the two NEURON-inbuilt HH channels for simplicity. As such, the output model will not be able to accurately reproduce the experimental traces, but the optimisation will demonstrate its ability to produce at least a spiking model. 

In [2]:
import json

from bluepyemodel.emodel_pipeline.emodel_pipeline import EModel_pipeline
from bluepyemodel.efeatures_extraction.targets_configurator import TargetsConfigurator

In [ ]:
# Clear any existing checkpoints to avoid conflicts with previous runs
!rm -r ./checkpoints

## Setting up the pipeline

The pipeline setup involves six key steps: **extraction** of e-features from electrophysiological recordings; **optimisation** of a NEURON cell model based on these e-features; storing the optimised model parameters; **validating** the models against specified protocols; and **plotting** the results, including traces, e-feature scores, and parameter distributions.

The [recipes.json](./config/recipes.json) file (displayed below) contains the key settings for the various stages of the e-model building pipeline.

* **``morph_path``**: Directory path for morphologies
* **``morphology``**: Contains a list with the morphology's arbitrary name and file name, located in `morph_path`.
* **``params``**: Specifies mechanisms, locations, distributions, and parameters.
* **``features``**: Path to the file with extraction outputs.
- **`pipeline_settings`**: Sets up the pipeline with several configuration options, including:  
  - `extract_absolute_amplitudes`: Set to `true` for performing optimisation independently of rheobase (threshold current).  
  - `optimiser`: Specifies the optimisation algorithm, set to `"SO-CMA"` (a single-objective Covariance Matrix Adaptation algorithm).  
  - `max_ngen`: Defines the maximum number of generations for the optimisation process, set to `5`.  
  - `optimisation_params`: Additional optimisation parameters, such as `offspring_size` set to `20`, indicating the number of solutions generated per generation.  
  - `validation_protocols`: Lists protocols used for validation, e.g., `["IDrest_0.4"]`.  
  - `morph_modifiers`: Set to an empty list `[]`, meaning no specific modifications to morphologies are applied by default.
  - `plot_currentscape`: We set this to False because we are using NEURON's built-in HH mechanism, where the currents are not defined as RANGE variables. As a result, Currentscape is unable to access and plot these currents.

In [3]:
recipes_path = "./config/recipes.json"
with open(recipes_path, 'r') as file:
    recipe = json.load(file)

print(json.dumps(recipe, indent=4))

{
    "simplecell": {
        "morph_path": "./morphologies/",
        "morphology": [
            [
                "simple",
                "simple.swc"
            ]
        ],
        "params": "config/params/simple.json",
        "features": "config/features/simplecell.json",
        "pipeline_settings": {
            "path_extract_config": "config/extract_config/simplecell_config.json",
            "extract_absolute_amplitudes": true,
            "optimiser": "SO-CMA",
            "max_ngen": 5,
            "optimisation_params": {
                "offspring_size": 20
            },
            "validation_protocols": [
                "IDrest_0.4"
            ],
            "morph_modifiers": [],
            "plot_currentscape": false
        }
    }
}


To begin, we need to instantiate the pipeline using the ``EModel_pipeline`` class. This class is responsible for loading the ``recipes.json`` file and configuring the pipeline settings based on its content. The following are the minimal required parameters:

- **`emodel`**: Name of the e-model

- **`etype`**: electrical type, `"cADpyr"`, indicating continuous adapting pyramidal cells.

- **`species`**: Biological species, `"rat"`, for which the model is developed.

- **`brain_region`**: Target brain region, `"SSCX"` (somatosensory cortex).

- **`morphology`**: Filename of the morphology file located in the `./morphologies` folder, with a spherical shape characterized by a diameter and length of 10 micrometers.

- **`morphology_format`**: Specifies the format of the morphology file, "swc", with support for SWC, ASC formats.

In [4]:
emodel = "simplecell"
etype = "cADpyr"
species = "rat"
brain_region = "SSCX"
morphology = "simple"
morphology_format = "swc"

pipeline = EModel_pipeline(
    emodel=emodel,
    etype=etype,
    species=species,
    brain_region=brain_region,
    recipes_path=recipes_path,
    data_access_point="local",
)

## Extracting the features

We need to download the required data using the script `./download_ephys_data.sh`. This dataset features continuous adapting pyramidal cells (cADpyr) e-type models from the rat somatosensory cortex. The data can be obtained from this [repository](<https://github.com/BlueBrain/SSCxEModelExamples/tree/main/feature_extraction/input-traces/C060109A1-SR-C1>).

In [ ]:
!sh ./download_ephys_data.sh

In [6]:
filenames = [
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_326.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_327.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_328.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_329.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDrest_ch0_330.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_349.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_350.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_351.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_352.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IDthresh_ch0_353.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_266.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_267.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_268.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_269.ibw",
    "./ephys_data/C060109A1-SR-C1/X_IV_ch0_270.ibw"
]

We can now define the targets, which include the protocols (ecodes) and the features. We define two protocols: `IDrest` and `IV` protocols.

For the IDrest protocol, with an amplitude of 0.2 nA (and 0.4 nA for validation), we select Spikecount, mean_frequency, and voltage_base to capture the neuron’s spiking activity and resting potential. In the IV protocol, with an amplitude of -0.1 nA, voltage_base and ohmic_input_resistance_vb_ssse are used to assess the neuron’s baseline potential and input resistance

The `tolerance` of 0.1 allows for a range of ±0.1 around each target amplitude for extracting e-features. For example, if the target amplitude is 0.4, the acceptable range would be from 0.3 to 0.5 Values, excluding 0.3 and 0.5.

In [7]:
targets = {
    "IDrest": {
        "amplitudes": [0.2, 0.4],
        "efeatures": [
            "Spikecount",
            "mean_frequency",
            "voltage_base",
        ],
    },
    "IV": {
        "amplitudes": [-0.1],
        "efeatures": [
            "voltage_base",
            "ohmic_input_resistance_vb_ssse",
        ],
    }
}

tolerance = 0.1

The `ecodes_metadata` dictionary defines parameters for each protocol: `ljp` is the liquid junction potential correction (14.0 mV). A [LJP](https://en.wikipedia.org/wiki/Liquid_junction_potential) value is subtracted from each voltage value. Here, the membrane potential will be shifted in the hyperpolarising direction by 14 mV. `ton` and `toff` represent the start and stop times (in ms) for current injection.

In [8]:
ecodes_metadata = {
    "IDthresh": {"ljp": 14.0, "ton": 700, "toff": 2700},
    "IDrest": {"ljp": 14.0, "ton": 700, "toff": 2700},
    "IV": {"ljp": 14.0, "ton": 20, "toff": 1020},
}

We save this targets in an object called `ExtractionTargetConfigurator` (ETC), which will serve as the input for the feature extractor

In [9]:
files_metadata = []
for filename in filenames:
    fn = filename.split("/")[-1]
    for ecode in ecodes_metadata:
        if ecode in fn:
            files_metadata.append(
                {
                    "cell_name": filename.split("/")[-2],
                    "filename": filename.split("/")[-1].split(".")[0],
                    "ecodes": {ecode: ecodes_metadata[ecode]},
                    "other_metadata": {
                        "i_file": filename,
                        "v_file": filename.replace("ch0", "ch1"),
                        "i_unit": "A",
                        "v_unit": "V",
                        "t_unit": "s",
                    },
                }
            )


targets_formated = []
for ecode in targets:
    for amplitude in targets[ecode]["amplitudes"]:
        for efeature in targets[ecode]["efeatures"]:
            targets_formated.append(
                {
                    "efeature": efeature,
                    "protocol": ecode,
                    "amplitude": amplitude,
                    "tolerance": tolerance,
                }
            )


configurator = TargetsConfigurator(pipeline.access_point)
configurator.new_configuration(files_metadata, targets_formated)
configurator.save_configuration()

We can now proceed to extract the e-features

In [ ]:
pipeline.extract_efeatures()

The results of the feature extraction is stored in `./config/features/simplecell.json`. Let's take a look at the extracted features

In [11]:
fcc_path = "./config/features/simplecell.json"
with open(fcc_path, 'r') as file:
    fcc = json.load(file)

print(json.dumps(fcc, indent=4))

{
    "efeatures": [
        {
            "efel_feature_name": "Spikecount",
            "protocol_name": "IDrest_0.2",
            "recording_name": "soma.v",
            "threshold_efeature_std": null,
            "default_std_value": 0.001,
            "mean": 5.0,
            "original_std": 3.605551275463989,
            "sample_size": 4,
            "efeature_name": "Spikecount",
            "weight": 1.0,
            "efel_settings": {
                "interp_step": 0.025,
                "strict_stiminterval": true,
                "Threshold": -48.99374771118164
            }
        },
        {
            "efel_feature_name": "mean_frequency",
            "protocol_name": "IDrest_0.2",
            "recording_name": "soma.v",
            "threshold_efeature_std": null,
            "default_std_value": 0.001,
            "mean": 3.9105121810617045,
            "original_std": 1.2910808300604681,
            "sample_size": 3,
            "efeature_name": "mean_frequency",
   

The "validation": true field indicates the IDrest_0.4 protocol will only be used for validation

## Setting the parameter for the optimisation

The parameters for the optimisation are defined in `./config/simple.json`. In this case, we are optimising two parameters: ``gnabar_hh`` and ``gkbar_hh``. These parameters determine the maximum conductances for sodium and potassium ion channels in a Hodgkin-Huxley neuron model, with optimisation ranges of 0.05–0.125 (S/cm<sup>2</sup>) and 0.01–0.075 (S/cm<sup>2</sup>), respectively.

In [12]:
emc_path = "./config/params/simple.json"
with open(emc_path, 'r') as file:
    emc = json.load(file)

print(json.dumps(emc, indent=4))

{
    "mechanisms": {
        "somatic": {
            "mech": [
                "hh"
            ]
        }
    },
    "distributions": {},
    "parameters": {
        "__comment": "define constants as single values and params to optimise as tuples of bounds: [lower, upper]",
        "global": [
            {
                "name": "v_init",
                "val": -80
            },
            {
                "name": "celsius",
                "val": 34
            }
        ],
        "somatic": [
            {
                "name": "Ra",
                "val": 100
            },
            {
                "name": "cm",
                "val": 1
            },
            {
                "name": "ena",
                "val": 50
            },
            {
                "name": "ek",
                "val": -90
            },
            {
                "name": "gnabar_hh",
                "val": [
                    0.05,
                    0.125
                ]
  

## Running the optimisation

We can now run the optimisation process using the `EModel_pipeline` class. The seed parameter is used to set the random number generator for reproducibility. The optimisation runs for 5 generations with a population size of 20. After each optimisation round is completed, the results are saved in a pickle file for each seed within the `./checkpoints` directory

In [ ]:
pipeline.optimise(seed=1)

## Validating the model

This validation step is optional but essential for verifying that the optimised model reliably represents biological behavior. It involves executing the protocols defined under the `validation_protocols` key in `recipes.json` to ensure the model's e-feature scores are within acceptable limits.

In [ ]:
pipeline.store_optimisation_results()
pipeline.validation()

The `./final.json` contains the best model across all seeds. Since we're only running a single seed here, the result will simply be that model. Now, let's check the final results

In [3]:
final_path = "./final.json"
with open(final_path, 'r') as file:
    final = json.load(file)

print(json.dumps(final, indent=4))

{
    "simplecell__1": {
        "emodel": "simplecell",
        "etype": "cADpyr",
        "ttype": null,
        "mtype": null,
        "species": "rat",
        "brain_region": "SSCX",
        "iteration": null,
        "synapse_class": null,
        "allen_notation": null,
        "score": 510.2861723258447,
        "parameters": {
            "gnabar_hh.somatic": 0.05000000003749999,
            "gkbar_hh.somatic": 0.07499999996750001
        },
        "fitness": {
            "IDrest_0.2.soma.v.Spikecount": 1.1094003924504583,
            "IDrest_0.2.soma.v.mean_frequency": 250.0,
            "IDrest_0.2.soma.v.voltage_base": 6.86265875272435,
            "IV_-0.1.soma.v.voltage_base": 2.3141131806698945,
            "IV_-0.1.soma.v.ohmic_input_resistance_vb_ssse": 250.0
        },
        "features": {
            "IDrest_0.2.soma.v.Spikecount": 1.0,
            "IDrest_0.2.soma.v.mean_frequency": 1739.1304358817733,
            "IDrest_0.2.soma.v.voltage_base": -82.86184220245

- **`score`**: The global z-score for the optimized model, summing z-scores of all e-features used in optimisation (e.g., `507.86`).

- **`parameters`**: Best hall of fame parameters of the optimised e-model

- **`fitness`**: Z-scores for each optimized e-feature, indicating how closely the model matches experimental data. High scores (e.g., `250.0`) suggest a poor fit.

- **`features`**: The actual numerical values of each e-feature produced by the model.

- **`validation_fitness`**: Z-scores for validation e-features; empty (`{}`) here, indicating no validation recorded.

- **`validated`**: Shows validation status. Here, `false` means the model didn't pass, as fitness scores must be below 5 to pass.


## Plotting the results

The plots are stored in the `./figures/` directory, organised into subfolders: `efeatures_extraction` for e-feature figures by protocol, distributions for parameter distributions, `optimisation` for optimisation curves and progress, `parameter_evolution` for parameter changes over generations, `scores` for z-scores of optimised e-features, `traces` for optimised e-model traces.

In [ ]:
pipeline.plot()